In [ ]:
# TotalJobs UK Data Engineer Job Analysis

# 📥 Step 1: Load and Inspect Data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

# Load data
df = pd.read_csv("../data/de_totaljobs_test2.csv")

# Inspect structure
print(df.info())
print(df.head(2))

# 🗂️ Check missing values
df.isna().sum()

# 🧼 Clean: Drop NaNs in location-related column
df.dropna(subset=['unitary_authority'], inplace=True)

# 🕰️ Convert publish date to datetime
df['published_at'] = pd.to_datetime(df['published_at'])
df['week'] = df['published_at'].dt.strftime('%Y-%U')

# 📊 Weekly Job Postings Distribution
plt.figure(figsize=(12, 6))
df['week'].value_counts().sort_index().plot(kind='bar')
plt.title("Weekly Distribution of Job Postings")
plt.xlabel("Week")
plt.ylabel("Number of Jobs")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 📍 Map: Jobs by Region
region_counts = df['unitary_authority'].value_counts().reset_index()
region_counts.columns = ['region', 'job_count']
print(region_counts.head())

# 🌍 Load UK administrative boundaries (GeoJSON or SHP file required)
uk_map = gpd.read_file("../data/uk_unitary_authorities.geojson")
uk_map = uk_map.merge(region_counts, left_on="CTYUA24NM", right_on="region", how="left")
uk_map['job_count'] = uk_map['job_count'].fillna(0)

# Plot map
uk_map.plot(column='job_count', cmap='Blues', edgecolor='black', legend=True, figsize=(10, 12))
plt.title("Job Distribution by Region")
plt.axis('off')
plt.show()

# 🔍 Extract Skills from Job Content
import re
from collections import Counter

skills_list = ['Python', 'SQL', 'Azure', 'AWS', 'GCP', 'Spark', 'Databricks']
def extract_skills(text):
    return [skill for skill in skills_list if re.search(fr'\b{re.escape(skill)}\b', str(text), re.IGNORECASE)]

all_skills = df['job_content'].apply(extract_skills)
flat_skills = [skill for sublist in all_skills for skill in sublist]
skills_freq = Counter(flat_skills)

# Plot Top Skills
pd.Series(skills_freq).sort_values(ascending=True).plot(kind='barh', figsize=(8, 6), color='teal')
plt.title("Top Technical Skills in Data Engineer Job Descriptions")
plt.xlabel("Frequency")
plt.tight_layout()
plt.show()

# 📌 Save clean data
df.to_csv("../data/de_cleaned.csv", index=False)
